In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Lambda, Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
import tensorflow as tf

from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os
import pandas as pd

In [2]:
df = pd.read_csv('test_assignment_sim.csv')

In [3]:
all_sites = ['SITE_0', 'SITE_1',
       'SITE_2', 'SITE_3', 'SITE_4', 'SITE_5', 'SITE_6', 'SITE_7', 'SITE_8',
       'SITE_9', 'SITE_10', 'SITE_11', 'SITE_12', 'SITE_13', 'SITE_14',
       'SITE_15', 'SITE_16', 'SITE_17', 'SITE_18', 'SITE_19', 'SITE_20',
       'SITE_21', 'SITE_22', 'SITE_23', 'SITE_24', 'SITE_25', 'SITE_26',
       'SITE_27', 'SITE_28', 'SITE_29', 'SITE_30', 'SITE_31', 'SITE_32',
       'SITE_33', 'SITE_34', 'SITE_35', 'SITE_36', 'SITE_37', 'SITE_38',
       'SITE_39', 'SITE_40', 'SITE_41', 'SITE_42', 'SITE_43', 'SITE_44',
       'SITE_45', 'SITE_46', 'SITE_47', 'SITE_48']

In [4]:
features = ['FLOWFACTOR', 'SPACING', 'DEP TIME']
F = []
for i in range(0, len(df)):
    samp = np.array(df[features ].iloc[0])
    F.append(samp)
F = np.array(F)
F = np.float32(F)

In [5]:
X = []
for i in range(0, len(df)):
    samp = np.array(df[all_sites].iloc[0])
    X.append(samp)
X = np.array(X)
X = np.float32(X)

In [6]:
tool = tf.keras.utils.to_categorical(df['TOOL'], dtype='float32')
tool = tool[:, 1:]

In [7]:
# reparameterization trick
# instead of sampling from Q(z|X), sample epsilon = N(0,I)
# z = z_mean + sqrt(var) * epsilon
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.

    # Arguments
        args (tensor): mean and log of variance of Q(z|X)

    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [19]:
# network parameters
input_shape = (49, )
intermediate_dim = 16
batch_size = 128
latent_dim = 3
epochs = 10

In [14]:
# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()
# plot_model(encoder, to_file='vae_mlp_encoder.png', show_shapes=True)

# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = Dense(49, activation='sigmoid')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
# decoder.summary()
# plot_model(decoder, to_file='vae_mlp_decoder.png', show_shapes=True)

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')


models = (encoder, decoder)

In [20]:
reconstruction_loss = binary_crossentropy(inputs, outputs)

reconstruction_loss *= 49
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
# vae.summary()

TypeError: unhashable type: 'DictWrapper'

In [21]:
vae.summary()

Model: "vae_mlp"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 49)]         0                                            
__________________________________________________________________________________________________
encoder (Functional)            [(None, 3), (None, 3 902         encoder_input[0][0]              
__________________________________________________________________________________________________
decoder (Functional)            (None, 49)           897         encoder[0][2]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 16)           800         encoder_input[0][0]              
____________________________________________________________________________________________

In [30]:
original_dim = 49
# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = Dense(original_dim, activation='sigmoid')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

models = (encoder, decoder)

# VAE loss = xent_loss + kl_loss
reconstruction_loss = binary_crossentropy(inputs,
                                          outputs)

reconstruction_loss *= original_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

# train the autoencoder
vae.fit(X_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, None))

Epoch 1/10


ValueError: in user code:

    C:\Users\Khamir\anaconda3\envs\tf\lib\site-packages\tensorflow\python\keras\engine\training.py:803 train_function  *
        return step_function(self, iterator)
    C:\Users\Khamir\anaconda3\envs\tf\lib\site-packages\tensorflow\python\keras\engine\training.py:793 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Khamir\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:969 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Khamir\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2352 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Khamir\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2711 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Khamir\anaconda3\envs\tf\lib\site-packages\tensorfl

In [31]:
# MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

image_size = x_train.shape[1]
original_dim = image_size * image_size
x_train = np.reshape(x_train, [-1, original_dim])
x_test = np.reshape(x_test, [-1, original_dim])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# network parameters
input_shape = (original_dim, )
intermediate_dim = 512
batch_size = 128
latent_dim = 2
epochs = 10 # Original code = 50

# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = Dense(original_dim, activation='sigmoid')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

models = (encoder, decoder)
data = (x_test, y_test)

# VAE loss = xent_loss + kl_loss
reconstruction_loss = binary_crossentropy(inputs,
                                          outputs)

reconstruction_loss *= original_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

# train the autoencoder
vae.fit(x_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, None))

Epoch 1/10
469/469 [==============================] - 4s 9ms/step - loss: 498.8506 - val_loss: 481.9449
Epoch 2/10
469/469 [==============================] - 4s 9ms/step - loss: 477.0258 - val_loss: 473.1595
Epoch 3/10
469/469 [==============================] - 4s 9ms/step - loss: 470.7032 - val_loss: 468.7222
Epoch 4/10
469/469 [==============================] - 4s 9ms/step - loss: 466.6763 - val_loss: 464.1683
Epoch 5/10
469/469 [==============================] - 4s 9ms/step - loss: 463.5248 - val_loss: 462.8653
Epoch 6/10
469/469 [==============================] - 4s 8ms/step - loss: 462.2646 - val_loss: 460.1258
Epoch 7/10
469/469 [==============================] - 4s 9ms/step - loss: 458.8471 - val_loss: 457.6156
Epoch 8/10
469/469 [==============================] - 4s 9ms/step - loss: 457.3793 - val_loss: 455.8767
Epoch 9/10
469/469 [==============================] - 4s 9ms/step - loss: 454.4875 - val_loss: 453.2520
Epoch 10/10
469/469 [==============================] - 4s 9ms/st

In [44]:
encoder.predict(w)

ValueError: in user code:

    C:\Users\Khamir\anaconda3\envs\tf\lib\site-packages\tensorflow\python\keras\engine\training.py:1428 predict_function  *
        return step_function(self, iterator)
    C:\Users\Khamir\anaconda3\envs\tf\lib\site-packages\tensorflow\python\keras\engine\training.py:1418 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Khamir\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:969 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Khamir\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2352 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Khamir\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2711 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Khamir\anaconda3\envs\tf\lib\site-packages\tens

In [50]:
w= x_train[0].transpose(0)

In [51]:
w.shape

(784,)

In [17]:
X_train = X[:400]
X_test = X[400:]

In [23]:
vae.fit(X_train,
        epochs=epochs,
        batch_size=batch_size,
       )

Epoch 1/10
4/4 [==============================] - 0s 1000us/step - loss: nan - kl_loss: nan - mse_loss: nan
Epoch 2/10
4/4 [==============================] - 0s 1ms/step - loss: nan - kl_loss: nan - mse_loss: nan
Epoch 3/10
4/4 [==============================] - 0s 1ms/step - loss: nan - kl_loss: nan - mse_loss: nan
Epoch 4/10
4/4 [==============================] - 0s 1ms/step - loss: nan - kl_loss: nan - mse_loss: nan
Epoch 5/10
4/4 [==============================] - 0s 1ms/step - loss: nan - kl_loss: nan - mse_loss: nan
Epoch 6/10
4/4 [==============================] - 0s 999us/step - loss: nan - kl_loss: nan - mse_loss: nan
Epoch 7/10
4/4 [==============================] - 0s 1ms/step - loss: nan - kl_loss: nan - mse_loss: nan
Epoch 8/10
4/4 [==============================] - 0s 1000us/step - loss: nan - kl_loss: nan - mse_loss: nan
Epoch 9/10
4/4 [==============================] - 0s 1ms/step - loss: nan - kl_loss: nan - mse_loss: nan
Epoch 10/10
4/4 [==============================

In [ ]:
X_test.shape